# 1. 問題の分析

## 競技プログラミング視点

* **最速**はハッシュ（値→添字）の1パス探索。各要素 `x` に対し、補数 `target-x` を先に見たか即時判定で **O(n)**。
* **一意解**が保証されるため、見つかったら即 return 可。安定した定数因子で高速。

## 業務開発視点

* 入力検証を入れて異常系を早期検知（型・長さ・値域）。
* 命名は `seen`（value→index Map）を使用。例外はテスト容易性のため **TypeError / RangeError** を明確化。

## JavaScript特有の考慮点

* **V8 最適化**: 数値単型配列前提・`for` ループ・`Map` を再利用し hidden class を安定化。
* **GC対策**: ループ内で不要オブジェクト生成をしない。`Map#get` 結果を一時変数に保持。
* **配列操作特性**: `for` インデックス走査が最も安定。`forEach` はコールバック割当の分だけ不利。

---

# 2. アルゴリズムアプローチ比較

| アプローチ                 | 時間計算量      | 空間計算量 | JS実装コスト | 可読性 | 備考                      |
| --------------------- | ---------- | ----- | ------- | --- | ----------------------- |
| 方法A（ハッシュ1パス）          | **O(n)**   | O(n)  | 低       | 中   | 値→添字 `Map`。一意解即時 return |
| 方法B（ソート＋二分/二-pointer） | O(n log n) | O(n)  | 中       | 高   | 元添字保持のためペア配列作成が必要       |
| 方法C（二重ループ）            | O(n²)      | O(1)  | 低       | 高   | 小規模のみ妥当、Follow-up 非充足   |

---

# 3. 選択したアルゴリズムと理由

* **選択**: 方法A（ハッシュ1パス）
* **理由**: Follow-up の「O(n²) より良く」を満たし、実装が簡潔で定数因子も小さい
* **JS最適化ポイント**:

  * `for (let i = 0; i < n; i++)` の単純走査
  * ループ内は `Map#get` → 比較 → `Map#set` の固定順
  * 一時配列/オブジェクト生成を抑制、例外はホットパス外（冒頭）で判定

---

# 4. コード実装（solution.js｜LeetCode 形式）

* 実行環境: **JavaScript (Node.js v22.14.0)**
* Module: **CommonJS**
* 外部ライブラリ: **不使用（Node標準のみ）**
* 実行: `node solution.js`（ローカル検証用に `module.exports` も付与。LeetCode では無視されます）

```javascript
'use strict';

/**
 * Two Sum（Pure Function）
 * 値→添字を保持する1パスのハッシュ法で解く。
 *
 * @param {number[]} nums - 整数配列（単型 number 前提）
 * @param {number} target - 目標和
 * @returns {[number, number]} - 条件を満たす 2 要素の添字（任意順）
 * @throws {TypeError} 配列/数値型でない、または有限数でない場合
 * @throws {RangeError} 配列長や値域が制約外、あるいは解が存在しない場合
 * @complexity 時間 O(n), 空間 O(n)
 */
var twoSum = function (nums, target) {
  // --- 入力検証（軽量 & 早期） ---
  if (!Array.isArray(nums)) throw new TypeError('nums must be an array');
  if (typeof target !== 'number' || !Number.isFinite(target)) {
    throw new TypeError('target must be a finite number');
  }
  const n = nums.length;
  // 問題制約: 2 <= n <= 1e4
  if (n < 2 || n > 1e4) throw new RangeError('nums length is out of range [2, 1e4]');

  // 値域チェック（-1e9..1e9）
  for (let i = 0; i < n; i++) {
    const v = nums[i];
    if (typeof v !== 'number' || !Number.isFinite(v)) {
      throw new TypeError('nums must contain only finite numbers');
    }
    if (v < -1e9 || v > 1e9) {
      throw new RangeError('nums[i] is out of allowed range [-1e9, 1e9]');
    }
  }
  if (target < -1e9 || target > 1e9) {
    // 厳密には問題は target にも値域があるため
    throw new RangeError('target is out of allowed range [-1e9, 1e9]');
  }

  // --- 本処理：ハッシュ1パス ---
  // seen: 値 -> その値を最初に見つけた添字
  /** @type {Map<number, number>} */
  const seen = new Map();

  for (let i = 0; i < n; i++) {
    const x = nums[i];
    const need = target - x;
    const j = seen.get(need);
    // Map#get は存在しない場合 undefined を返す
    if (j !== undefined) {
      // 一意解前提につき即 return（添字の順不同）
      return [j, i];
    }
    // 同じ要素を2回使わないため、未登録時のみ格納（最初の添字を固定化）
    if (!seen.has(x)) seen.set(x, i);
  }

  // 問題文上は常に解が存在するが、検証用途に例外を残す
  throw new RangeError('No valid two-sum solution found');
};

// --- ローカル実行サポート（LeetCode では無視される） ---
module.exports = twoSum;

// // 参考：ローカルでの簡易確認（コメント解除して利用）
// // console.log(twoSum([2,7,11,15], 9)); // [0,1]
// // console.log(twoSum([3,2,4], 6));     // [1,2]
// // console.log(twoSum([3,3], 6));       // [0,1]

Analyze Complexity
Runtime 1 ms
Beats 87.91%
Memory 56.86 MB
Beats 41.70%

```
---

# 5. 追加メモ（JS最適化チェックリスト）

* ループは **`for` 基本**。`forEach/map` は不要なコールバック割当で遅くなる。
* `Map` は数値キーに強く、`Object` よりキー変換コストが少ない（文字列化不要）。
* 例外はホットパス外（冒頭の検証）に寄せる。
* 配列は **単型（number のみ）** を維持して hidden class の安定性を確保。
* 一時オブジェクトや配列の生成を避ける（`need`・`j` 程度のスカラのみ）。
* ソート案（O(n log n)）は元添字保持のため追加配列が必要になり、今回は採用しない。

この問題はすでに**定数因子**での勝負なので、伸びしろは小さいですが、**LeetCodeの実行環境（ジャッジ専用）での微調整**はまだできます。要点だけ絞って提案します。

---

## 改善ポイント（実務版 → ジャッジ最適化版）

* **入力検証の削減**：LeetCodeは制約前提なので、重めの型/範囲チェックは外すと定数因子ぶん速く・軽くなります。
* **`Map` → `Object.create(null)`**：数値キーが文字列化されるデメリットはありますが、プロトタイプを持たないプレーン辞書は**メモリのオーバーヘッドが小さめ**になりやすいです（キー数が多いと顕著）。

  * `in` 判定も `Object.create(null)` なら安全（継承プロパティなし）。
  * `undefined` を未登録の番人値として使えるため追加の `has`/`delete` が不要。
* **1パス & 1回参照**：`get→has` の二度引きを避け、**1回の読み出し結果で分岐**する現行方針は最適なので維持。

---

## ジャッジ最適化版（LeetCodeフォーマット／CommonJS・外部Libなし）

```javascript
'use strict';

/**
 * @param {number[]} nums
 * @param {number} target
 * @return {[number, number]}
 * @complexity 時間 O(n), 空間 O(n)
 */
var twoSum = function(nums, target) {
  // Object.create(null) で原型チェーンを断ち、軽量辞書を確保
  const seen = Object.create(null); // value(string化) -> index

  // 1パス：未登録なら格納、補数があれば即返す
  for (let i = 0, n = nums.length; i < n; i++) {
    const x = nums[i];
    const need = target - x;

    // 未登録は undefined（番人値）になる
    const j = seen[need];
    if (j !== undefined) return [j, i];

    // 最初の出現だけ保持（同値重複時に初出を優先）
    if (seen[x] === undefined) seen[x] = i;
  }

  // 問題の前提では到達しない
  return [-1, -1];
};

module.exports = twoSum;

Analyze Complexity
Runtime 1 ms
Beats 87.91%
Memory 57.14 MB
Beats 35.94%

```

### 期待効果

* **Runtime**：入力検証・`Map` の内部オーバーヘッド分が減り、1ms帯の安定性が上がる可能性
* **Memory**：`Map` よりもやや低く出ることが多い（キー数が1e4近いケースで差が出やすい）

---

## もう一段の微調整（好みで選択）

* **`in` を使う版**（可読性重視）

  ```js
  if (need in seen) return [seen[need], i];
  if (!(x in seen)) seen[x] = i;
  ```

  `Object.create(null)` 前提なので継承衝突なし。`in` は存在判定に明示的で、`undefined` 番人に依存しないスタイル。

* **`Map` のまま行く派**
  V8の`Map`は数値キー最適化が効きやすく、実測で速いケースもあるため、**環境依存**で逆転します。もし `Object` 版で悪化するなら元の `Map` 実装に戻すのが正解です（ベンチ前提）。

---

## まとめ

* すでに **ほぼ上限近い性能**です。
* **ジャッジ専用なら**上記「Object辞書・検証削減」で**僅かなRuntime安定化とMemory微減**が狙えます。
* 逆に**業務コード**としては、これまでの**入力検証あり・Map版**のほうが堅牢です。用途に応じて使い分けがベストです。
